In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

from src.data.make_dataset import DICT_VARIABLES
from src.features import build_features

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

HOSPITAL_A_ANALIZAR = 112103  # Este es el Torax

## 1. Análisis Sociodemográfico de Pacientes Atendidos en el Hospital del Tórax

En este cuadernillo se quiere ver el perfil sociodemográfico de los pacientes hospitalizados
en el Hospital del Tórax. Se utilizará la base de datos pública de egresos hospitalarios DEIS
entre 2001 y 2020. Las preguntas que se quieren responder son:

- ¿Cómo es la distribución geográfica de los pacientes atendidos en el INT?
- ¿Cuál es el rango etario más atendido en el INT?
- ¿Cuál es el género más atendido en el INT?
- ¿Cuál es la previsión más atendida en el INT?
- ¿Cuál es la letra de FONASA más atendida en el INT?

Para responder estas preguntas se realizará una agrupación de los datos con todas las variables
sociodemográficas de interés para el análisis.

In [29]:
# Define todas las variables que se quieren saber

VARIABLES_SOCIODEMOGRAFICAS_ESTATICAS = ["ANO_EGRESO", "REGION_RESIDENCIA", "COMUNA_RESIDENCIA"]
VARIABLES_SOCIODEMOGRAFICAS_DINAMICAS = ["SEXO", "EDAD_CATEGORIA", "PREVISION", "BENEFICIARIO"]
VARIABLE_A_CONTAR = "DIAG1"

agrupacion_total = (
    VARIABLES_SOCIODEMOGRAFICAS_ESTATICAS
    + VARIABLES_SOCIODEMOGRAFICAS_DINAMICAS
    + [VARIABLE_A_CONTAR]
)

Luego, se genera la consulta de la base de datos. Para reducir el coste computacional, se filtra
la base de datos únicamente para los datos del INT. Posteriormente, se agrupa la base de datos
con las variables sociodemográficas de interés y se cuenta la ocurrencia de cada grupo.

In [30]:
desglose_sociodemografico = (
    pl.scan_csv("../data/processed/egresos_procesados.csv")
    .filter(pl.col("ESTABLECIMIENTO_SALUD") == HOSPITAL_A_ANALIZAR)
    .group_by(agrupacion_total)
    .count()
    .sort(["ANO_EGRESO"])
    .collect(streaming=True)
)

In [36]:
display(desglose_sociodemografico.head())

ANO_EGRESO,REGION_RESIDENCIA,COMUNA_RESIDENCIA,SEXO,EDAD_CATEGORIA,PREVISION,BENEFICIARIO,DIAG1,count
i64,i64,i64,str,str,str,str,str,u32
2001,13,13605,"""Mujer""","""(70, 80]""","""FONASA""","""B""","""K810""",1
2001,13,13124,"""Mujer""","""(40, 50]""","""ISAPRE""",null,"""I10X""",1
2001,13,13122,"""Hombre""","""(40, 50]""","""FONASA""","""D""","""I509""",2
2001,13,13123,"""Hombre""","""(80, 90]""","""DESCONOCIDO""",null,"""C950""",1
2001,13,13115,"""Hombre""","""(60, 70]""","""FONASA""","""C""","""Q859""",1


En los resultados se indica el perfil sociodemográfico para cada combinación de variable presente
en la base de datos. A modo de ejemplo, hubo 1 ocurrencia en el 2001 de una paciente de la Región
número 13 y Comuna 13605 que tenía entre 70 a 80 años que tenía PREVISION FONASA Letra B diagnosticada
con el diagnóstico K810.

Uno de los diagnósticos de interés para el Instituto es el de código "C341".

In [41]:
display(desglose_sociodemografico.filter(pl.col("DIAG1") == "C341"))
print(desglose_sociodemografico.filter(pl.col("DIAG1") == "C341").select(pl.col("count").sum()))

ANO_EGRESO,REGION_RESIDENCIA,COMUNA_RESIDENCIA,SEXO,EDAD_CATEGORIA,PREVISION,BENEFICIARIO,DIAG1,count
i64,i64,i64,str,str,str,str,str,u32
2001,13,13113,"""Mujer""","""(60, 70]""","""FONASA""","""B""","""C341""",1
2001,13,13101,"""Hombre""","""(50, 60]""","""FONASA""","""A""","""C341""",1
2001,8,8101,"""Hombre""","""(60, 70]""","""DESCONOCIDO""",null,"""C341""",1
2001,3,3102,"""Hombre""","""(50, 60]""","""FONASA""","""A""","""C341""",1
2001,13,13128,"""Mujer""","""(60, 70]""","""FONASA""","""B""","""C341""",1
2001,3,3101,"""Hombre""","""(60, 70]""","""FONASA""","""C""","""C341""",1
2001,13,13106,"""Mujer""","""(60, 70]""","""FONASA""","""B""","""C341""",1
2003,13,13110,"""Mujer""","""(50, 60]""","""FONASA""","""B""","""C341""",1
2003,6,6101,"""Hombre""","""(50, 60]""","""FONASA""","""A""","""C341""",1


shape: (1, 1)
┌───────┐
│ count │
│ ---   │
│ u32   │
╞═══════╡
│ 1709  │
└───────┘


Por lo tanto, se muestra cada desglose sociodemográfico de los pacientes atendidos. Además,
se indica que la suma de cada conteo de grupo suma 1709 entre 2001 y 2020. Esto está en concordancia
con los egresos totales (1709) para el diagnóstico mencionado en el periodo mencionado.